In [1]:
import random

import numpy as np
import scipy as sp

import scipy.linalg
import scipy.optimize

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

In [2]:
from markovc import *

In [3]:
from data import *

In [7]:
alpha = 0.3
x = [alpha, 1-alpha, alpha]
Q = create_rate_matrix_from_vector(x)

P = embedded_jump_chain(Q)
pi = equi_dtmc(P)

In [8]:
equi_deriv_ctmc(Q, P, pi, (1,2))

array([ 0.05625879,  0.18284107, -0.23909986])

In [9]:
def nll(xhat, data):
    Qhat = create_rate_matrix_from_vector(xhat)
    
    s = 0
    for (winner, choices), count in data.items():
#         print("{} {} {}".format(winner, choices, count))
        winner_idx = [idx for idx, val in enumerate(choices) if winner==val][0]
        Qs = submatrix(Qhat, choices)
        try:
            piS = equi_ctmc(Qs)[winner_idx]
        except ValueError:
            print(xhat)
            raise
        
#         print(piS)

        s += count * np.log(np.abs(piS))    
        
    return -s

In [10]:
data = gen_data(Q, 5000)
x0 = [random.random() for _ in range(len(x))]

res = sp.optimize.minimize(lambda x: nll(x, data), x0, method="L-BFGS-B", bounds=[(0.0001, 0.9999) for _ in range(len(x0))])
print(res)

   status: 0
      nit: 10
      fun: 3765.3226003263685
  success: True
     nfev: 48
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
        x: array([ 0.27963076,  0.71659801,  0.28063702])
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00245564, -0.0039563 , -0.0003638 ])


In [11]:
xopt = res['x']
Qhat = create_rate_matrix_from_vector(xopt)

In [12]:
Qhat

array([[-0.99622878,  0.27963076,  0.71659801],
       [ 0.72036924, -1.00100625,  0.28063702],
       [ 0.28340199,  0.71936298, -1.00276497]])

In [17]:
Phat = embedded_jump_chain(Qhat)
pihat = equi_ctmc(Qhat)

In [20]:
equi_deriv_ctmc(Qhat, Phat, pihat, (2, 1))

array([-0.06042023, -0.17736484,  0.23778507])

In [16]:
nll(xopt, data)

3765.3226003263685

In [ ]:
gen_data(Q, n=100, pinc=0.2)

In [ ]:
n = 3

In [ ]:
m = n * (n-1) / 2

In [ ]:
import math

In [ ]:
int(1/2 + math.sqrt(1/4 + 2 * m)+0.5)